In [17]:
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
from langchain.document_loaders import PyPDFLoader
def load_pdf_files(files):
    # Assuming the provided 'files' list contains URLs
    pdf_url = files[0]

    # Read PDF content from the URL
    pdf_loader = PyPDFLoader(pdf_url)
    docs = pdf_loader.load_and_split()

    # If there are more files in the list, process them
    for i in range(1, len(files)):
        pdf_url = files[i]
        pdf_loader = PyPDFLoader(pdf_url)
        docs += pdf_loader.load_and_split()

    return docs

In [19]:
files=["./DataSourceFiles/IPC_186045.pdf"]
# files=["D:/Files/LLM/Project/DataSourceFiles/IPC_186045.pdf","D:\Files\LLM\Src_docs\special_marriage_act.pdf","D:\Files\LLM\Src_docs\THE_CODE_OF_CIVIL_PROCEDURE_1908.pdf","D:\Files\LLM\Src_docs\THE_LAND_ACQUISITION_ACT.pdf"]
docs=load_pdf_files(files)
print("docs in PDF files: ",len(docs))

docs in PDF files:  187


In [20]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# def split_docs(documents,chunk_size=1000,chunk_overlap=20):
#   text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
#   docs = text_splitter.split_documents(documents)
#   return docs

# splited_docs = split_docs(docs)
# print(len(splited_docs))


In [35]:
from langchain.vectorstores import FAISS
from langchain.embeddings import  HuggingFaceInstructEmbeddings,HuggingFaceHubEmbeddings,HuggingFaceEmbeddings

# HFIembeddings = HuggingFaceInstructEmbeddings(model_name="WhereIsAI/UAE-Large-V1")    
# HFIembeddings = HuggingFaceHubEmbeddings(repo_id="sentence-transformers/all-mpnet-base-v2")
HFIembeddings = HuggingFaceHubEmbeddings(repo_id="sentence-transformers/all-MiniLM-L6-v2")
# HFIembeddings = HuggingFaceInstructEmbeddings(model_name="thenlper/gte-small",cache_folder="./Models/")
# HFIembeddings = HuggingFaceEmbeddings(model_name="D:\Files\LLM\Project\Models\sentence-transformers_all-MiniLM-L6-v2",cache_folder="./Models/")
vectorstore = FAISS.from_documents(docs,embedding=HFIembeddings)
print("vectore store created")

You're using a different task than the one specified in the repository. Be sure to know what you're doing :)


vectore store created


In [36]:
# from langchain.embeddings import SentenceTransformerEmbeddings
# embeddings = SentenceTransformerEmbeddings(
#     model_name="D:\Files\LLM\Project\Models\sentence-transformers_all-MiniLM-L6-v2",
#     model_kwargs={'device': 'cuda'},
#     encode_kwargs={'normalize_embeddings': True},
#     # query_instruction="Represent this sentence for searching relevant passages: "
# )

# from langchain.vectorstores import Chroma
# vectorstore = Chroma.from_documents(splited_docs, embeddings)

In [37]:
# from langchain.vectorstores import Chroma
# # from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
# # from langchain.embeddings import  HuggingFaceInstructEmbeddings,HuggingFaceHubEmbeddings,HuggingFaceEmbeddings

# from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
# from sentence_transformers import SentenceTransformer
# # model = SentenceTransformer(model_name_or_path="sentence-transformers/all-MiniLM-L6-v2",cache_folder="./Models/")
# sentence_transformer_ef = SentenceTransformerEmbeddingFunction(model_name="D:\Files\LLM\Project\Models\sentence-transformers_all-MiniLM-L6-v2")
# # # HFIembeddings = HuggingFaceHubEmbeddings(repo_id="sentence-transformers/all-mpnet-base-v2")
# # # # embedding_function = SentenceTransformerEmbeddings(model_name="gte-small") #,cache_folder="./Models/")
# # print((sentence_transformer_ef))
# vectorstore = Chroma.from_documents(documents=pages,embedding=sentence_transformer_ef,collection_name="law_collection")

In [38]:
from langchain.prompts import PromptTemplate
template = """
You're a helpful AI assistant assigned to assist individuals seeking legal advice within the framework of Indian laws and the constitution.
Your role is to guide users through legal processes and provide information in a lawful manner.
Answer questions step by step, highlighting relevant sections of Indian laws and the constitution.
Refrain from responding to queries that may not contribute to legal affairs, and provide accurate information without distortion.
Include reference/source information such as section number and all related details.

{context}

Question: {question} 
Answer:
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [39]:

from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
chain = RetrievalQA.from_chain_type(
    llm = HuggingFaceHub(repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1", model_kwargs={"temperature":0.8, "max_length":4096,"max_new_tokens":4096}),
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),    
    # return_source_documents=True,
    chain_type_kwargs={"prompt": prompt,"verbose": True},
)

In [43]:
# q="what is the punishment for robbery ?"
q="who is judge ?"
# q="one person has commited defamation, what should be punishment for it?"
# q="rioting has happened in neighbouring society. who will be responsible and what is the punishment for it"
# q="what is punishment for robbery and also murder ?"
# q="what are the rights of tenant and landlord ?"
# q="what is biscuit ?"
# q="person is selling adulterated drugs which is harmful to the health of people. person has also sold drug to the children below age of 15 what will be punishment for it?"
# q+=" with section number. add all details."
# q+=" including section number and all related details."
response = chain({"query":q, "early_stopping":True,"min_length":2000,"max_tokens":5000})
# print(response)
from write_in_file import generate_docx_with_bullets
# from data_extracter import extract_data_from_response
# result,src_data,src_pg_nms=extract_data_from_response(response)
# generate_docx_with_bullets(heading=q,main_paragraph=response["result"],srcs=src_pg_nms,output_folder="./tmp/")
generate_docx_with_bullets(heading=q,main_paragraph=response["result"],output_folder="./tmp/")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You're a helpful AI assistant assigned to assist individuals seeking legal advice within the framework of Indian laws and the constitution.
Your role is to guide users through legal processes and provide information in a lawful manner.
Answer questions step by step, highlighting relevant sections of Indian laws and the constitution.
Refrain from responding to queries that may not contribute to legal affairs, and provide accurate information without distortion.
Include reference/source information such as section number and all related details.

17 
 1[17 “Government ”.—The word “Government ” denotes the Central Government or t he Government 
of a 2***State.]  
3[18. “India ”.—“India ” means the territory of India excluding t he State of Jammu and Kashmir.]  
19. “Judge ”.—The word “Judge ” denotes not only every person who is officially designated as a 
Judge, but also every perso

In [41]:
# def write_in_notepad(response):
#     from write_in_file import open_in_notepad
#     open_in_notepad(response["result"])


